In [ ]:
from core.utils import Tibanna
from core import ff_utils
from datetime import datetime

# set enviroment and key/connection
env = 'fourfront-webdev'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

def get_wfr_report(wfr):
    """For a given workflow_run_sbg item, grabs details, uuid, run_status, wfr name, date, and run time"""
    wfr_data= ff_utils.get_metadata(wfr , connection=ff)
    wfr_uuid = wfr_data['uuid']
    wfr_status = wfr_data['run_status']
    wfr_name = wfr_data['title'].split(' run ')[0]
    wfr_time = datetime.strptime(wfr_data['date_created'],'%Y-%m-%dT%H:%M:%S.%f+00:00')
    run_hours = (datetime.now()-wfr_time).total_seconds()/3600
    wfr_rep = {'wfr_uuid': wfr_data['uuid'],
               'wfr_status': wfr_data['run_status'],
               'wfr_name': wfr_data['title'].split(' run ')[0],
               'wfr_date': wfr_time,
               'run_time': run_hours}
    return wfr_rep

# what kind of files should be searched for worflow run inputs, use url compatible naming
raw_file_types = ['files-fastq']

# checked workflows
workflow_names = ['md5', 'fastqc-0-11-4-1/1']

patched_md5 = 0
patched_fastqc = 0

fastqc_status =[]
# get all files for fastqc
files = ff_utils.get_metadata('files-fastq' , connection=ff)['@graph']
for raw_file in files:
    wfr_report = []
    wfrs = raw_file.get('workflow_run_inputs')
    # get a report on all workflow_runs
    if wfrs:
        for wfr in wfrs:
            wfr_report.append(get_wfr_report(wfr))  
    # sort the report by date and name
    wfr_report = sorted(wfr_report, key=lambda k: (k['wfr_date'], k['wfr_name']))
    for wf_name in workflow_names:
        sub_list = [i for i in wfr_report if i['wfr_name'] == wf_name]
        if len(sub_list) > 1:
            print "multiple", wf_name, "in file", raw_file['uuid']
            print "please run 'Find old workflow_runs on raw_files and change status to deleted' for clean-up"
        elif sub_list:
            run_status = sub_list[0]['wfr_status']
            wfr_uuid = sub_list[0]['wfr_uuid']

            if wf_name == 'md5':
                if raw_file['status']== 'uploaded' and raw_file.get('md5sum') and raw_file['content_md5sum']:
                    if run_status == "output_file_transfer_finished":
                        patch_data = {'run_status': "complete"}
                        ff_utils.patch_metadata(patch_data, obj_id=wfr_uuid ,connection=ff)
                        patched_md5 += 1

            if wf_name == 'fastqc-0-11-4-1/1':
                fastqc_status.append(run_status)
                if raw_file['status']== 'uploaded' and raw_file.get('quality_metric'):
                    if run_status == "output_file_transfer_finished":
                        patch_data = {'run_status': "complete"}
                        ff_utils.patch_metadata(patch_data, obj_id=wfr_uuid ,connection=ff)
                        patched_fastqc += 1

print str(patched_md5), 'md5 wfrs patched with complete'
print str(patched_fastqc), 'fastqc wfrs patched with complete'            